In [ ]:
install.packages("data.table")
install.packages(c("rpart","rpart.plot"), , dependencies = TRUE)
install.packages("lpSolve", dependencies = TRUE)
library("data.table")
library("lpSolve")
library("rpart")
library("rpart.plot")

## Data preparation 

Read the data

In [2]:
mydata= read.csv(file.choose())

In [3]:
colnames(mydata)

[1] "ï..Item."    "Total.sales" "hour.1"      "hour.2"      "hour.3"     
 [6] "hour.4"      "hour.5"      "hour.6"      "hour.7"      "hour.8"     
[11] "hour.9"      "hour.10"     "hour.11"     "hour.12"     "hour.13"    
[16] "hour.14"     "hour.15"     "hour.16"     "hour.17"     "hour.18"    
[21] "hour.19"     "hour.20"     "hour.21"     "hour.22"     "hour.23"    
[26] "hour.24"

In [4]:
nrow(mydata)

[1] 2446

Calculate the total inventory used (share of sale)

In [5]:
Total = rowSums(mydata[,3:26])
head(Total)

[1] 0.933 0.938 0.996 0.842 1.000 0.998

Adding "Total" to main table

In [6]:
mydata = cbind(mydata, Total)
head(mydata)

ï..Item.,Total.sales,hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,...,hour.16,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24,Total
1,2854,0.113,0.066,0.064,0.064,0.016,0.052,0.042,0.029,...,0.038,0.027,0.034,0.027,0.004,0.006,0.044,0.039,0.001,0.933
2,2817,0.116,0.051,0.025,0.011,0.072,0.038,0.009,0.046,...,0.049,0.047,0.042,0.035,0.041,0.013,0.032,0.012,0.029,0.938
3,5089,0.118,0.032,0.072,0.040,0.071,0.018,0.051,0.024,...,0.029,0.048,0.039,0.015,0.037,0.048,0.036,0.033,0.018,0.996
4,6906,0.138,0.016,0.019,0.048,0.027,0.010,0.015,0.006,...,0.028,0.027,0.037,0.029,0.046,0.046,0.034,0.003,0.045,0.842
5,3115,0.116,0.032,0.048,0.054,0.035,0.049,0.023,0.075,...,0.048,0.039,0.044,0.012,0.004,0.000,0.000,0.000,0.000,1.000
6,4404,0.122,0.052,0.068,0.081,0.006,0.083,0.048,0.080,...,0.026,0.029,0.049,0.030,0.012,0.005,0.008,0.000,0.000,0.998


Finding stockouts and non-stockouts

In [7]:
stockouts=mydata[(mydata$Total>=1 & mydata$hour.24==0),]
not_stockouts=mydata[!(mydata$Total>=1 & mydata$hour.24==0),]
head(stockouts)

,ï..Item.,Total.sales,hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,...,hour.16,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24,Total
5,5,3115,0.116,0.032,0.048,0.054,0.035,0.049,0.023,0.075,...,0.048,0.039,0.044,0.012,0.004,0.000,0.000,0,0,1.000
12,12,4943,0.144,0.060,0.038,0.019,0.076,0.072,0.008,0.000,...,0.036,0.021,0.028,0.023,0.019,0.000,0.000,0,0,1.002
14,14,4783,0.132,0.033,0.002,0.083,0.033,0.068,0.057,0.006,...,0.030,0.043,0.043,0.004,0.011,0.031,0.000,0,0,1.000
17,17,6294,0.114,0.060,0.031,0.078,0.082,0.026,0.081,0.022,...,0.044,0.041,0.035,0.028,0.009,0.010,0.015,0,0,1.000
19,19,4716,0.144,0.046,0.048,0.004,0.030,0.056,0.036,0.053,...,0.029,0.034,0.039,0.013,0.028,0.036,0.000,0,0,1.001
36,36,3206,0.133,0.003,0.080,0.025,0.005,0.056,0.073,0.008,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,1.000


In [8]:
nrow(stockouts)

[1] 534

In [9]:
nrow(not_stockouts)

[1] 1912

Calculating the cumulative sheet

In [29]:
a=stockouts[,3:26]
b=setDT(a)

In [11]:
head(b)

hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,hour.9,hour.10,...,hour.15,hour.16,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24
0.116,0.032,0.048,0.054,0.035,0.049,0.023,0.075,0.063,0.082,...,0.038,0.048,0.039,0.044,0.012,0.004,0.000,0.000,0,0
0.144,0.060,0.038,0.019,0.076,0.072,0.008,0.000,0.079,0.081,...,0.028,0.036,0.021,0.028,0.023,0.019,0.000,0.000,0,0
0.132,0.033,0.002,0.083,0.033,0.068,0.057,0.006,0.079,0.049,...,0.039,0.030,0.043,0.043,0.004,0.011,0.031,0.000,0,0
0.114,0.060,0.031,0.078,0.082,0.026,0.081,0.022,0.054,0.027,...,0.023,0.044,0.041,0.035,0.028,0.009,0.010,0.015,0,0
0.144,0.046,0.048,0.004,0.030,0.056,0.036,0.053,0.076,0.048,...,0.049,0.029,0.034,0.039,0.013,0.028,0.036,0.000,0,0
0.133,0.003,0.080,0.025,0.005,0.056,0.073,0.008,0.029,0.067,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0


In [12]:
b[, names(a) := Reduce(`+`, b, accumulate = TRUE)]

In [13]:
head(b)

hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,hour.9,hour.10,...,hour.15,hour.16,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24
0.116,0.148,0.196,0.250,0.285,0.334,0.357,0.432,0.495,0.577,...,0.853,0.901,0.940,0.984,0.996,1.000,1.000,1.000,1.000,1.000
0.144,0.204,0.242,0.261,0.337,0.409,0.417,0.417,0.496,0.577,...,0.875,0.911,0.932,0.960,0.983,1.002,1.002,1.002,1.002,1.002
0.132,0.165,0.167,0.250,0.283,0.351,0.408,0.414,0.493,0.542,...,0.838,0.868,0.911,0.954,0.958,0.969,1.000,1.000,1.000,1.000
0.114,0.174,0.205,0.283,0.365,0.391,0.472,0.494,0.548,0.575,...,0.818,0.862,0.903,0.938,0.966,0.975,0.985,1.000,1.000,1.000
0.144,0.190,0.238,0.242,0.272,0.328,0.364,0.417,0.493,0.541,...,0.822,0.851,0.885,0.924,0.937,0.965,1.001,1.001,1.001,1.001
0.133,0.136,0.216,0.241,0.246,0.302,0.375,0.383,0.412,0.479,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [14]:
stockouts_cumulative=as.data.frame(b)
nrow(stockouts_cumulative)

[1] 534

Finding the stockout time

In [30]:
stockout_time=vector(mode="numeric")
for (i in 1:nrow(stockouts_cumulative)){
stockout_time[i]=match(max(stockouts_cumulative[i,]),stockouts_cumulative[i,])
}


Adding stockout time to table

In [16]:
stockouts=cbind(stockouts,stockout_time)
head(stockouts)

,ï..Item.,Total.sales,hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,...,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24,Total,stockout_time
5,5,3115,0.116,0.032,0.048,0.054,0.035,0.049,0.023,0.075,...,0.039,0.044,0.012,0.004,0.000,0.000,0,0,1.000,20
12,12,4943,0.144,0.060,0.038,0.019,0.076,0.072,0.008,0.000,...,0.021,0.028,0.023,0.019,0.000,0.000,0,0,1.002,20
14,14,4783,0.132,0.033,0.002,0.083,0.033,0.068,0.057,0.006,...,0.043,0.043,0.004,0.011,0.031,0.000,0,0,1.000,21
17,17,6294,0.114,0.060,0.031,0.078,0.082,0.026,0.081,0.022,...,0.041,0.035,0.028,0.009,0.010,0.015,0,0,1.000,22
19,19,4716,0.144,0.046,0.048,0.004,0.030,0.056,0.036,0.053,...,0.034,0.039,0.013,0.028,0.036,0.000,0,0,1.001,21
36,36,3206,0.133,0.003,0.080,0.025,0.005,0.056,0.073,0.008,...,0.000,0.000,0.000,0.000,0.000,0.000,0,0,1.000,14


## Clustering

In [17]:
clusters= kmeans(mydata[,3:26], 3)
centroids= clusters$centers
centroids

hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,hour.9,hour.10,...,hour.15,hour.16,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24
0.1254305,0.04359736,0.04466032,0.04384334,0.04414641,0.04369253,0.04095168,0.04040556,0.04180381,0.04472035,...,0.03470571,0.03523719,0.03391069,0.03233968,0.021487555,0.019650073,0.016916545,0.014503660,0.012000000,0.0090937042
0.1263434,0.04305051,0.04401515,0.04057576,0.04084848,0.04123232,0.04307071,0.04324242,0.03980808,0.04421212,...,0.02637374,0.02315152,0.01987879,0.01465657,0.006964646,0.005393939,0.003681818,0.002671717,0.001641414,0.0008838384
0.1249335,0.04111949,0.04131182,0.04094952,0.04152843,0.04155655,0.04206006,0.04151949,0.04150032,0.04005112,...,0.03485942,0.03538403,0.03468307,0.03502045,0.024482428,0.024197444,0.023484984,0.022796805,0.020881789,0.0176198083


In [18]:
head(clusters$cluster)

[1] 3 3 3 3 1 3

### Demand Estimation

Find lost sales and calculate true demand

In [19]:
true_demand= vector(mode = "numeric")
for (i in 1:nrow(stockouts)) {
    index=stockouts[i,1]
    determined_cluster=clusters$cluster[index]
    lost_sales= sum(centroids[determined_cluster, (stockouts[i,28]+1):24])
    true_demand[i]= stockouts[i,2]/(1-lost_sales)
}


Adding true demand to stockouts

In [20]:
stockouts=cbind(stockouts, true_demand)
head(stockouts)

,ï..Item.,Total.sales,hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,...,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24,Total,stockout_time,true_demand
5,5,3115,0.116,0.032,0.048,0.054,0.035,0.049,0.023,0.075,...,0.044,0.012,0.004,0.000,0.000,0,0,1.000,20,3287.647
12,12,4943,0.144,0.060,0.038,0.019,0.076,0.072,0.008,0.000,...,0.028,0.023,0.019,0.000,0.000,0,0,1.002,20,5400.907
14,14,4783,0.132,0.033,0.002,0.083,0.033,0.068,0.057,0.006,...,0.043,0.004,0.011,0.031,0.000,0,0,1.000,21,5095.336
17,17,6294,0.114,0.060,0.031,0.078,0.082,0.026,0.081,0.022,...,0.035,0.028,0.009,0.010,0.015,0,0,1.000,22,6429.625
19,19,4716,0.144,0.046,0.048,0.004,0.030,0.056,0.036,0.053,...,0.039,0.013,0.028,0.036,0.000,0,0,1.001,21,4890.074
36,36,3206,0.133,0.003,0.080,0.025,0.005,0.056,0.073,0.008,...,0.000,0.000,0.000,0.000,0.000,0,0,1.000,14,3583.316


In [21]:
true_demand = not_stockouts$Total.sales
not_stockouts= cbind(not_stockouts, true_demand)
head(not_stockouts)

,ï..Item.,Total.sales,hour.1,hour.2,hour.3,hour.4,hour.5,hour.6,hour.7,hour.8,...,hour.17,hour.18,hour.19,hour.20,hour.21,hour.22,hour.23,hour.24,Total,true_demand
1,1,2854,0.113,0.066,0.064,0.064,0.016,0.052,0.042,0.029,...,0.027,0.034,0.027,0.004,0.006,0.044,0.039,0.001,0.933,2854
2,2,2817,0.116,0.051,0.025,0.011,0.072,0.038,0.009,0.046,...,0.047,0.042,0.035,0.041,0.013,0.032,0.012,0.029,0.938,2817
3,3,5089,0.118,0.032,0.072,0.040,0.071,0.018,0.051,0.024,...,0.048,0.039,0.015,0.037,0.048,0.036,0.033,0.018,0.996,5089
4,4,6906,0.138,0.016,0.019,0.048,0.027,0.010,0.015,0.006,...,0.027,0.037,0.029,0.046,0.046,0.034,0.003,0.045,0.842,6906
6,6,4404,0.122,0.052,0.068,0.081,0.006,0.083,0.048,0.080,...,0.029,0.049,0.030,0.012,0.005,0.008,0.000,0.000,0.998,4404
7,7,5950,0.147,0.012,0.051,0.013,0.071,0.041,0.007,0.052,...,0.045,0.049,0.048,0.034,0.002,0.035,0.004,0.023,0.856,5950


In [22]:
Demands= rbind(stockouts[,c(1,29)], not_stockouts[,c(1,28)])
head(Demands)

,ï..Item.,true_demand
5,5,3287.647
12,12,5400.907
14,14,5095.336
17,17,6429.625
19,19,4890.074
36,36,3583.316


In [23]:
tail(Demands)

,ï..Item.,true_demand
2440,2440,3737
2441,2441,4307
2442,2442,4105
2443,2443,3498
2444,2444,5201
2446,2446,4027


## Demand prediction 

import new data from products features and historical prices

In [24]:
mydata2= read.csv(file.choose())

In [25]:
colnames(mydata2)

[1] "ï..Item."                           "Price"                             
 [3] "Beginning_of_Season"                "Weekend"                           
 [5] "Event_Length"                       "Morning"                           
 [7] "Branded"                            "Color_Popularity"                  
 [9] "ConcurrentEvents"                   "Number_Competing_Styles_in_Event"  
[11] "Relative_Price_of_Competing_Styles" "Num_Branded_Events12"              
[13] "Brand_MSRP_Index"                   "Bottoms"                           
[15] "Tops"                               "Dresses"                           
[17] "Demand"

In [26]:
head(mydata2)

ï..Item.,Price,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses,Demand
1,31,0,0,1,1,1,0.02,3,29,0.95,0,1.31,0,0,1,2854
2,33,0,0,2,1,1,0.11,3,9,1.00,2,0.64,0,0,1,2817
3,29,1,0,2,1,1,0.11,4,38,0.88,4,0.82,0,1,0,5089
4,28,0,0,3,1,1,0.31,3,13,1.01,2,0.62,0,0,0,6906
5,39,1,0,2,1,0,0.31,4,25,0.84,0,0.70,0,0,0,3376
6,28,0,0,3,0,0,0.08,5,26,0.43,0,0.78,0,1,0,4647


In [27]:
prediction_data= merge(Demands, mydata2, by = "ï..Item.")
head(prediction_data)

ï..Item.,true_demand,Price,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses,Demand
1,2854.000,31,0,0,1,1,1,0.02,3,29,0.95,0,1.31,0,0,1,2854
2,2817.000,33,0,0,2,1,1,0.11,3,9,1.00,2,0.64,0,0,1,2817
3,5089.000,29,1,0,2,1,1,0.11,4,38,0.88,4,0.82,0,1,0,5089
4,6906.000,28,0,0,3,1,1,0.31,3,13,1.01,2,0.62,0,0,0,6906
5,3287.647,39,1,0,2,1,0,0.31,4,25,0.84,0,0.70,0,0,0,3376
6,4404.000,28,0,0,3,0,0,0.08,5,26,0.43,0,0.78,0,1,0,4647


In [28]:
prediction_data= prediction_data[, 2:17]
head(prediction_data)

true_demand,Price,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses
2854.000,31,0,0,1,1,1,0.02,3,29,0.95,0,1.31,0,0,1
2817.000,33,0,0,2,1,1,0.11,3,9,1.00,2,0.64,0,0,1
5089.000,29,1,0,2,1,1,0.11,4,38,0.88,4,0.82,0,1,0
6906.000,28,0,0,3,1,1,0.31,3,13,1.01,2,0.62,0,0,0
3287.647,39,1,0,2,1,0,0.31,4,25,0.84,0,0.70,0,0,0
4404.000,28,0,0,3,0,0,0.08,5,26,0.43,0,0.78,0,1,0


In [29]:
set.seed(10)

p_integer = sample(nrow(prediction_data), size=0.8 * nrow(prediction_data))

Train_set <- prediction_data[p_integer,]
Test_set <- prediction_data[-p_integer,]

### Regression Tree

In [31]:
regression_tree= rpart(formula = true_demand~., data = Train_set, method = "anova")
tree_Demand_pred= predict(regression_tree, Test_set)
regression_tree

n= 1956 

node), split, n, deviance, yval
      * denotes terminal node

  1) root 1956 7854284000  4735.091  
    2) Price>=30.5 787  101860100  3332.392 *
    3) Price< 30.5 1169 5161479000  5679.423  
      6) Brand_MSRP_Index>=0.525 1059 3565263000  5525.487  
       12) Event_Length< 3.5 1047 3333541000  5490.740  
         24) Branded< 0.5 143  249173200  4779.782 *
         25) Branded>=0.5 904 3000652000  5603.204  
           50) Relative_Price_of_Competing_Styles>=0.63 852 2516672000  5527.573 *
           51) Relative_Price_of_Competing_Styles< 0.63 52  399257000  6842.384  
            102) Color_Popularity< 0.255 43  191937100  6260.163 *
            103) Color_Popularity>=0.255 9  123102100  9624.103 *
       13) Event_Length>=3.5 12  120165800  8557.163 *
      7) Brand_MSRP_Index< 0.525 110 1329532000  7161.405  
       14) Number_Competing_Styles_in_Event< 44 100  594364400  6556.599  
         28) Brand_MSRP_Index>=0.31 87  336633700  6198.162 *
         29) Brand_MSR

## Linear Regression

In [32]:
lm_model = lm(true_demand ~ ., data = Train_set)
lm_Demand_pred = predict(lm_model, Test_set)
lm_model


Call:
lm(formula = true_demand ~ ., data = Train_set)

Coefficients:
                       (Intercept)                               Price  
                         11620.830                            -201.730  
               Beginning_of_Season                             Weekend  
                            82.137                              34.827  
                      Event_Length                             Morning  
                           397.011                            -731.263  
                           Branded                    Color_Popularity  
                           653.936                             801.348  
                  ConcurrentEvents    Number_Competing_Styles_in_Event  
                            13.831                               2.812  
Relative_Price_of_Competing_Styles                Num_Branded_Events12  
                          -855.880                              32.841  
                  Brand_MSRP_Index                    

## Comparing the RMSE

In [62]:
lm_rmse = sqrt(mean((Test_set$true_demand - lm_Demand_pred)^2))
tree_rmse = sqrt(mean((Test_set$true_demand - tree_Demand_pred)^2))

printf <- function(...) cat(sprintf(...))
printf("The RMSE for Linear Regression: %f   RMSE for Desicion Tree: %f.", round(lm_rmse,2) , round(tree_rmse,2))

The RMSE for Linear Regression: 1508.850000   RMSE for Desicion Tree: 1393.300000.

#### Therefore, the decision tree regression model is the preferred option due to its lower error (RMSE) when compared to linear regression.

## Price Optimization 

$$
\begin{aligned}
\operatorname{Max} & \sum_{i \in N} \sum_{j \in M} p_j \widehat{D}_{i j k} x_{i j} \\
\text { s.t } \\
& \sum_{j \in M} x_{i j}=1 \quad \forall i \in N \\
& \sum_{i \in N} \sum_{j \in M} p_j x_{i j}=k \\
& x_{i j} \in\{0,1\}\\
\end{aligned}
$$

Importing Data test 

In [34]:
Data_test= read.csv(file.choose())
head(Data_test)

ï..,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses
Item A,1,0,2,0,1,0.17,2,3,2,0.87,0,1,0
Item B,1,0,2,0,0,0.65,2,3,1,0.33,0,1,0
Item C,1,0,2,0,1,0.08,2,3,1,1.18,0,1,0


 Defining list of prices

In [35]:
Prices= c(25, 30, 35)
P = rep(Prices, nrow(Data_test))
P

[1] 25 30 35 25 30 35 25 30 35

Preparing Variables

In [36]:
Data_test2= Data_test[rep(seq_len(nrow(Data_test)), each = 3), ]
Data_test2

,ï..,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses
1,Item A,1,0,2,0,1,0.17,2,3,2,0.87,0,1,0
1.1,Item A,1,0,2,0,1,0.17,2,3,2,0.87,0,1,0
1.2,Item A,1,0,2,0,1,0.17,2,3,2,0.87,0,1,0
2,Item B,1,0,2,0,0,0.65,2,3,1,0.33,0,1,0
2.1,Item B,1,0,2,0,0,0.65,2,3,1,0.33,0,1,0
2.2,Item B,1,0,2,0,0,0.65,2,3,1,0.33,0,1,0
3,Item C,1,0,2,0,1,0.08,2,3,1,1.18,0,1,0
3.1,Item C,1,0,2,0,1,0.08,2,3,1,1.18,0,1,0
3.2,Item C,1,0,2,0,1,0.08,2,3,1,1.18,0,1,0


Note that since k is a part of constraints and there are different values for k for different choices of prices. **Thus, we need to solve the model several time**

In [37]:
possible_k= seq(length(Prices)*min(Prices), length(Prices)*max(Prices), by=5)
possible_k

[1]  75  80  85  90  95 100 105

Initialization

In [53]:
Demand_pred_tree= vector(mode= "numeric")
Demand_pred_lm= vector(mode= "numeric")
Objectives= vector(mode= "numeric")
Solutions= matrix(nrow= length(possible_k), ncol= length(Prices)*nrow(Data_test))

### Optimal Prices for Decision Tree

In [54]:
for (n in 1:length(possible_k)){
    for (i in 1:length(P)){
        Data_test2$Price= P[i]
        Data_test2$Relative_Price_of_Competing_Styles= P[i]/(possible_k[n]/3)
        Demand_pred_tree[i]= predict(regression_tree, Data_test2[i,])
    }
    Obj_coeff= Demand_pred_tree*P
    Cons_coeff= matrix(c(1,1,1,0,0,0,0,0,0,
                         0,0,0,1,1,1,0,0,0,
                         0,0,0,0,0,0,1,1,1,
                         P[1],P[2],P[3],P[4],P[5],P[6],P[7],P[8],P[9]), nrow=4, byrow=TRUE)
    Dir = c("==",
            "==",
            "==",
            "==")
    RHS= c(1,1,1, possible_k[n])
    Model= lp("max", Obj_coeff, Cons_coeff, Dir, RHS, all.bin= TRUE)
    Objectives[n]= Model$objval
    Solutions[n,]= Model$solution
}
Demand_pred_tree

[1] 5527.573 5527.573 3332.392 6198.162 6198.162 3332.392 5527.573 5527.573
[9] 3332.392

In [55]:
results_tree = cbind(Solutions, Objectives)
tree_best_objective = max(Objectives)
tree_solution = results_tree[match(tree_best_objective, Objectives),]
tree_best_objective
tree_solution

[1] 517599.3

0.0        1.0        0.0        0.0        1.0        0.0        0.0 
                      Objectives 
       1.0        0.0   517599.3

### Optimal Price for Linear Regression

In [56]:
for (n in 1:length(possible_k)){
    for (i in 1:length(P)){
        Data_test2$Price= P[i]
        Data_test2$Relative_Price_of_Competing_Styles= P[i]/(possible_k[n]/3)
        Demand_pred_lm[i]= predict(lm_model, Data_test2[i,])
    }
    Obj_coeff= Demand_pred_lm*P
    Cons_coeff= matrix(c(1,1,1,0,0,0,0,0,0,
                         0,0,0,1,1,1,0,0,0,
                         0,0,0,0,0,0,1,1,1,
                         P[1],P[2],P[3],P[4],P[5],P[6],P[7],P[8],P[9]), nrow=4, byrow=TRUE)
    Dir = c("==",
            "==",
            "==",
            "==")
    RHS= c(1,1,1, possible_k[n])
    Model= lp("max", Obj_coeff, Cons_coeff, Dir, RHS, all.bin= TRUE)
    Objectives[n]= Model$objval
    Solutions[n,]= Model$solution
}
Demand_pred_lm

[1] 7028.358 5897.437 4766.517 6959.169 5828.249 4697.328 6789.670 5658.750
[9] 4527.829

In [57]:
results_lm = cbind(Solutions, Objectives)
lm_best_objective = max(Objectives)
lm_solution = results_lm[match(lm_best_objective, Objectives),]
lm_solution

0.0        1.0        0.0        0.0        1.0        0.0        0.0 
                      Objectives 
       1.0        0.0   510528.9

#### So the optimal price does not change.

Solving the model for all possible values of k and with assuming item A cannot be sold at 25$.

In [64]:
Demand_pred= vector(mode= "numeric")
Objectives= vector(mode= "numeric")
Solutions= matrix(nrow= length(possible_k), ncol= length(Prices)*nrow(Data_test))


for (n in 1:length(possible_k)){
    for (i in 1:length(P)){
        Data_test2$Price= P[i]
        Data_test2$Relative_Price_of_Competing_Styles= P[i]/(possible_k[n]/3)
        Demand_pred[i]= predict(regression_tree, Data_test2[i,])
    }
    Obj_coeff= Demand_pred*P
    Cons_coeff= matrix(c(1,1,1,0,0,0,0,0,0,
                         0,0,0,1,1,1,0,0,0,
                         0,0,0,0,0,0,1,1,1,
                         1,0,0,0,0,0,0,0,0,
                         P[1],P[2],P[3],P[4],P[5],P[6],P[7],P[8],P[9]), nrow=5, byrow=TRUE)
    Dir = c("==",
            "==",
            "==",
            "==",
            "==")
    RHS= c(1,1,1, 0, possible_k[n])
    Model= lp("max", Obj_coeff, Cons_coeff, Dir, RHS, all.bin= TRUE)
    Objectives[n]= Model$objval
    Solutions[n,]= Model$solution
}
Demand_pred

[1] 5527.573 5527.573 3332.392 6198.162 6198.162 3332.392 5527.573 5527.573
[9] 3332.392

In [65]:
Solutions

0,0,0,0,0,0,0,0,0
0,1,0,1,0,0,1,0,0
0,1,0,0,1,0,1,0,0
0,1,0,0,1,0,0,1,0
0,1,0,0,1,0,0,0,1
0,0,1,0,1,0,0,0,1
0,0,1,0,0,1,0,0,1


In [66]:
Objectives

[1]      0.0 458970.6 489961.4 517599.3 468405.8 419212.3 349901.2

The first element of objective list is zero as the equation did not have a feasible solution.

In [67]:
cbind(Solutions, Objectives)

,,,,,,,,,Objectives
0,0,0,0,0,0,0,0,0,0.0
0,1,0,1,0,0,1,0,0,458970.6
0,1,0,0,1,0,1,0,0,489961.4
0,1,0,0,1,0,0,1,0,517599.3
0,1,0,0,1,0,0,0,1,468405.8
0,0,1,0,1,0,0,0,1,419212.3
0,0,1,0,0,1,0,0,1,349901.2


In [68]:
Solutions[match(max(Objectives),Objectives),]

[1] 0 1 0 0 1 0 0 1 0